In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from music21 import converter, instrument, note, chord, stream, midi
import numpy as np
import os
import random

In [ ]:
def get_notes():
    filepath="/content/drive/My Drive/new data set/reduce/"
    files=os.listdir(filepath)
    #file = "All The Small Things - Chorus.midi"
    Notes=[]
    for file in files:
      print(file)
      #try:
      st = converter.parse(filepath+file)#('/content/drive/My Drive/new data set/CANON1.mid')#(filepath+file)
      instru = instrument.partitionByInstrument(st)
      if instru:  
          notes = instru.parts[0].recurse()
      else:  
          notes = st.flat.notes
      for element in notes:
          if isinstance(element, note.Note):
              #if (element.volume != 0):
                Notes.append(str(element.pitch))#+str(element.duration))
                #print(element.pitch,element.volume)
          if isinstance(element, chord.Chord):
            #if (element.volume != 0):
              Notes.append(str(element.pitches[1]))
              #Notes.append(str('.'.join(str(n) for n in element.normalOrder)))#(element.normalOrder[0]))#('.'.join(str(n) for n in element.normalOrder))
              #print(element.volume)
      #except:
      #    pass
    with open('Note', 'a+')as f:
        f.write(str(Notes))
    offset = 0
    output_notes = []
    for data in Notes:
      if ('.' in data) or data.isdigit():
          notes_in_chord = data.split('.')
          notes = []
          for current_note in notes_in_chord:
              new_note = note.Note(int(current_note))
              new_note.storedInstrument = instrument.Piano()
              notes.append(new_note)
          new_chord = chord.Chord(notes)
          new_chord.offset = offset
          output_notes.append(new_chord)
      else:
          new_note = note.Note(data)
          new_note.offset = offset
          new_note.storedInstrument = instrument.Piano()
          output_notes.append(new_note)
      offset += 1

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='/content/drive/My Drive/check.mid')
    return Notes

In [ ]:
mid = midi.MidiFile()
filepath="/content/drive/My Drive/new data set/Canon/"
files=os.listdir(filepath)
file = files[6]
#for file in files:
mid.open(filepath+file)
mid.read()
mid.close()
#mid = MidiFile(filepath+file, clip=True)
print(file)
#print(mid)
print(len(mid.tracks))
#for track in mid.tracks:
#    print(track)
#for parameter1 in mid.tracks[0]:
  #print(msg)
for i in range(len(mid.tracks)-2):
  mid.tracks.remove(mid.tracks[2])
mid.tracks.remove(mid.tracks[0])
#mid = mid.tracks[1]
mid.open("/content/drive/My Drive/new data set/reduce/"+str("reduced_")+file, 'wb')
mid.write()
print(mid)
mid.close()
#mid.save('/content/drive/My Drive/reduce.midi')

IndexError: ignored

In [ ]:
# use bach_ave-maria testing
Train_note = get_notes()
print(Train_note[0:10])
print(len(Train_note)) # number of notes

reduced_CANON1.mid
reduced_pa-canon1.mid
reduced_pacanon-in-d-orch.mid
reduced_pcanon-m-dikmen.mid
reduced_pachcanon-piano-solo.mid
reduced_pa-canon2.mid
['D3', 'A3', 'D4', 'F#4', 'A2', 'C#4', 'E4', 'A4', 'B2', 'F#3']
5079


In [ ]:
# 編號
note_name = sorted(set(i for i in Train_note))
note_dict = dict((j,i) for i,j in enumerate(note_name))
print(note_dict)
print(len(note_dict)) # 不一樣的note數量
# embedding?? 需要? 只要output one hot encoding就好?

{'A2': 0, 'A3': 1, 'A4': 2, 'A5': 3, 'B2': 4, 'B3': 5, 'B4': 6, 'B5': 7, 'C#3': 8, 'C#4': 9, 'C#5': 10, 'C#6': 11, 'C2': 12, 'C3': 13, 'C4': 14, 'C5': 15, 'C6': 16, 'D2': 17, 'D3': 18, 'D4': 19, 'D5': 20, 'D6': 21, 'E2': 22, 'E3': 23, 'E4': 24, 'E5': 25, 'F#2': 26, 'F#3': 27, 'F#4': 28, 'F#5': 29, 'F#6': 30, 'F2': 31, 'F3': 32, 'F4': 33, 'F5': 34, 'G2': 35, 'G3': 36, 'G4': 37, 'G5': 38, 'G6': 39}
40


In [ ]:
from tensorflow import keras
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D 
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense 
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# 資料的x與y
sequence_length = 20 
predict_length = 1 #預測長度
network_input = []
network_output = []
index = list(range(len(Train_note) - sequence_length))
random.seed(20)
random.shuffle(index)
print(index)
for i in range(0, len(Train_note) - sequence_length):# - (predict_length-1)):
  sequence_in = Train_note[index[i]: index[i] + sequence_length]
  sequence_out = Train_note[index[i] + sequence_length]#:i + sequence_length + predict_length]
  network_input.append([note_dict[k] for k in sequence_in])
  #network_output.append([note_dict[k] for k in sequence_out])
  network_output.append(note_dict[sequence_out])

network_input = np.reshape(network_input, (len(network_input), sequence_length,1)) #reshape
#network_output = np.reshape(network_output, (len(network_output), predict_length,1)) #reshape
notes_len = len(set(Train_note))
network_input = network_input / (float(notes_len)-1) #使資料範圍在0~1
network_output = keras.utils.to_categorical(network_output) #有?種不同的輸出->形成?維資料 
#lambda network_input, network_output:(tf.one_hot(network_input, depth = notes_len), network_output)
print(np.shape(network_input))
print(np.shape(network_output))
print(network_input[100:110])
#print(network_output)

[4965, 197, 1022, 1284, 680, 4876, 2159, 180, 4101, 1247, 4980, 2637, 1791, 4513, 4995, 3285, 2917, 2528, 2651, 1435, 3545, 135, 4220, 513, 3934, 2067, 4664, 4541, 74, 295, 2289, 2268, 1149, 2534, 2030, 1372, 1490, 2992, 531, 1796, 1797, 4688, 3425, 1060, 3731, 3863, 2006, 3000, 4347, 4874, 3676, 4617, 3428, 4114, 1351, 2212, 4939, 1936, 357, 4801, 2000, 4496, 2506, 4004, 4842, 1301, 94, 1633, 1272, 4796, 196, 1512, 401, 1396, 4766, 912, 2375, 843, 138, 422, 4429, 750, 1179, 1897, 2161, 4473, 3771, 4585, 1031, 4529, 4646, 2895, 4739, 2292, 3516, 5030, 1193, 2287, 2081, 2980, 4315, 4050, 3102, 3712, 1973, 844, 2998, 3524, 1100, 142, 1452, 3456, 2223, 258, 3949, 3390, 4191, 249, 2425, 2802, 3287, 65, 1777, 3126, 2910, 2762, 2217, 3070, 4216, 1300, 2663, 4722, 3047, 4655, 4528, 3554, 4082, 3334, 3672, 498, 3313, 4750, 2085, 876, 2747, 1863, 4764, 856, 1667, 3718, 1234, 4381, 2860, 669, 412, 147, 3881, 20, 2850, 1780, 449, 2820, 1582, 3244, 1114, 105, 832, 2111, 2349, 622, 3389, 288, 3249,

In [ ]:
# Keras Sequential Model 
max_id = notes_len - 1
model = keras.models.Sequential([
  keras.layers.Bidirectional(keras.layers.LSTM(256,input_shape=[sequence_length, 1], return_sequences=True, dropout=0.2, recurrent_dropout=0.2)), #input_shape=[None, max_id]?
  # 缺attention layer
  keras.layers.LSTM(256),# return_sequences=True),
  #keras.layers.LSTM(256),# return_sequences=True),
  #keras.layers.Flatten(),
  keras.layers.Dense(256),
  keras.layers.Dense(notes_len),#keras.layers.TimeDistributed( #加入TimeDistributed的原因: 與其訓練模型來讓他在最後一個時步預測接下來的10個值，我們可以訓練她在美已個時步預測接下來的10個值
  keras.layers.Activation('softmax')
  ])

In [ ]:
model.build([None,sequence_length,1]) #input shape
model.summary()        # Model info

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_3 (Bidirection (None, 20, 512)           528384    
_________________________________________________________________
lstm_7 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_7 (Dense)              (None, 40)                10280     
_________________________________________________________________
activation_3 (Activation)    (None, 40)                0         
Total params: 1,391,912
Trainable params: 1,391,912
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizera = keras.optimizers.RMSprop(lr = 0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizera,metrics=["accuracy"]) 

In [ ]:
# 所有輸出在訓練期間都有用途，但只有最後一個十步的輸出可用來預測和評估，因此評估時使用自訂評量標準，只計算最後一個時步輸出的MSE
#def last_time_step_mse(Y_true,Y_pred):
#   return keras.metrics.mean_squared_error(Y_true[:,-1],Y_pred[:,-1])
history = model.fit(network_input, network_output, epochs=100, batch_size=64)  #, metrics=[last_time_step_mse]
model.save("/content/drive/My Drive/music_canon_long.h5") #/AI_CLASS/fruit/fruit.h5 #music_canon_test.h5

Epoch 1/100
80/80 [==============================] - 45s 493ms/step - loss: 3.3106 - accuracy: 0.0768
Epoch 2/100
80/80 [==============================] - 39s 491ms/step - loss: 3.1846 - accuracy: 0.0909
Epoch 3/100
80/80 [==============================] - 39s 492ms/step - loss: 3.1498 - accuracy: 0.0964
Epoch 4/100
80/80 [==============================] - 39s 491ms/step - loss: 3.1551 - accuracy: 0.0935
Epoch 5/100
80/80 [==============================] - 39s 491ms/step - loss: 3.1465 - accuracy: 0.1010
Epoch 6/100
80/80 [==============================] - 40s 494ms/step - loss: 3.1119 - accuracy: 0.1020
Epoch 7/100
80/80 [==============================] - 39s 490ms/step - loss: 3.1131 - accuracy: 0.1099
Epoch 8/100
80/80 [==============================] - 39s 490ms/step - loss: 3.1023 - accuracy: 0.1121
Epoch 9/100
80/80 [==============================] - 39s 492ms/step - loss: 3.1006 - accuracy: 0.1053
Epoch 10/100
80/80 [==============================] - 39s 491ms/step - loss: 3.083

In [ ]:
save_dict_to_file(note_dict,'/content/drive/My Drive/canon.npy') 
notedic = load_dict_from_file('/content/drive/My Drive/canon.npy')
print(notedic) # displays "world"
print(len(notedic))

{0: 'A2', 1: 'A3', 2: 'A4', 3: 'A5', 4: 'B2', 5: 'B3', 6: 'B4', 7: 'B5', 8: 'C#3', 9: 'C#4', 10: 'C#5', 11: 'C#6', 12: 'C2', 13: 'C3', 14: 'C4', 15: 'C5', 16: 'C6', 17: 'D2', 18: 'D3', 19: 'D4', 20: 'D5', 21: 'D6', 22: 'E2', 23: 'E3', 24: 'E4', 25: 'E5', 26: 'F#2', 27: 'F#3', 28: 'F#4', 29: 'F#5', 30: 'F#6', 31: 'F2', 32: 'F3', 33: 'F4', 34: 'F5', 35: 'G2', 36: 'G3', 37: 'G4', 38: 'G5', 39: 'G6'}
40


In [ ]:
model_load = keras.models.load_model('/content/drive/My Drive/music_canon_long.h5')
randindex = np.random.randint(0, len(network_input) - 1)
#notedic = dict((i,j) for i, j in enumerate(note_name))  
print(notedic)
pattern = network_input[23] #randindex#把某個input資料取出，作為預測的初始input
print(type(pattern))
prediction = []
for i in range(300):
    pattern1 = pattern[i:i+sequence_length]
    prediction_input = np.reshape(pattern1, (1, sequence_length, 1)) #pattern1#size:(1,20,1)
    #prediction_input = prediction_input / (float(notes_len)-1)
    prediction_output = model_load.predict(prediction_input, verbose=0) 
    index = np.argmax(prediction_output) #機率最大的
    result = notedic[index]    #其所代表的音符
    print(result)
    prediction.append(result)
    pattern = np.append(pattern,index/ (float(notes_len)-1))
# 生成mid音乐
# def create_music():
# prediction = generate_notes(model, network_input, note_name, notes_len)
offset = 0
output_notes = []
# 生成 Note（音符）或 Chord（和弦）对象
for data in prediction:
    if ('.' in data) or data.isdigit():
        notes_in_chord = data.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(data)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='/content/drive/My Drive/canon_long_output.mid')

{0: 'A2', 1: 'A3', 2: 'A4', 3: 'A5', 4: 'B2', 5: 'B3', 6: 'B4', 7: 'B5', 8: 'C#3', 9: 'C#4', 10: 'C#5', 11: 'C#6', 12: 'C2', 13: 'C3', 14: 'C4', 15: 'C5', 16: 'C6', 17: 'D2', 18: 'D3', 19: 'D4', 20: 'D5', 21: 'D6', 22: 'E2', 23: 'E3', 24: 'E4', 25: 'E5', 26: 'F#2', 27: 'F#3', 28: 'F#4', 29: 'F#5', 30: 'F#6', 31: 'F2', 32: 'F3', 33: 'F4', 34: 'F5', 35: 'G2', 36: 'G3', 37: 'G4', 38: 'G5', 39: 'G6'}
<class 'numpy.ndarray'>
A4
A4
B4
G4
F#4
F#4
F#3
B2
F#4
A3
D4
F#2
F#4
F#4
E4
E4
E4
C#5
A4
A4
A4
A4
G4
F#4
A3
C#4
E4
D5
E4
C#5
E4
E4
A5
A4
A5
A5
A3
C#4
A3
F#5
B5
A5
C#5
C#5
G2
B4
E4
B4
E4
D4
E4
C#4
C#5
A4
F#4
A4
C#5
E4
A4
D4
E4
D4
C#4
A4
A4
G4
F#4
D4
B4
D5
F#4
C#5
A5
F#4
G2
D4
E4
E4
C#5
C#5
A4
B4
B4
B4
A4
G4
F#4
B4
F4
F#4
C#5
C#5
D4
F#4
G4
A4
E4
A4
C#4
D4
B3
C#4
A4
F#4
A3
C#5
F#4
F#4
G3
B4
C#5
F#4
F#4
C#5
A4
C#5
F#5
C#4
F#4
A4
E4
F#4
D5
B4
B4
A4
G4
F#4
G4
D4
C#5
F#4
F#4
C#5
A4
D6
D6
F#4
F#4
E4
A4
C#5
E4
A4
G5
F#5
G4
E5
E5
F#5
F#5
E5
E5
C#5
A4
A4
A5
A5
B5
B5
B5
A5
A5
A5
A5
A5
B5
B5
B5
C#6
C#5
C#5

'/content/drive/My Drive/canon_long_output.mid'